In [1]:
import pandas as pd
import numpy as np
import sklearn 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
np.random.seed(23)
pd.set_option('display.max_columns', None)

C:\Users\natal\AppData\Local\Temp\ipykernel_15944\729211821.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
mobile_modelling = pd.read_csv('../Data/Data_modelling/mobile_modelling.csv')

X = mobile_modelling.iloc[:, 0:-1]
y = mobile_modelling.iloc[:, -1]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, stratify=y_val, test_size=0.3, random_state=42
)

mobile_df = pd.concat([X_train, y_train], axis=1)

print(mobile_df.shape)

X_train = mobile_df.iloc[:, 0:-1]
y_train = mobile_df.iloc[:, -1]

(980, 21)


We'll analyse some of the models with using 2 types of dataset transformations:
- Standarisation
- Box Cox transformation

### Standarisation

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)

In [12]:
# more than 0.02
columns_to_drop_1 = ['touch_screen', 'blue', 'dual_sim', 'four_g', 'wifi', 'three_g']
X_train_1 = X_train_scaled.drop(columns=columns_to_drop_1)
X_val_1 = X_val_scaled.drop(columns=columns_to_drop_1)

# more than 0.05
columns_to_drop_2 = ['touch_screen', 'blue', 'dual_sim', 'four_g', 'wifi', 'three_g', 'fc', 'n_cores', 'm_dep', 'sc_h', 'clock_speed', 'sc_w', 'pc', 'talk_time', 'int_memory', 'mobile_wt']
X_train_2 = X_train_scaled.drop(columns=columns_to_drop_2)
X_val_2 = X_val_scaled.drop(columns=columns_to_drop_2)

# just RAM
X_train_3 = X_train_scaled['ram']
X_val_3 = X_val_scaled['ram']

In [14]:
from sklearn.metrics import classification_report

### decision tree

In [15]:
from sklearn.tree import DecisionTreeClassifier,plot_tree

print("for all the columns")
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)
y_pred = tree.predict(X_val)
print(classification_report(y_val, y_pred))


print("for columns with importance over 0.02")
tree = DecisionTreeClassifier()
tree.fit(X_train_1,y_train)
y_pred_1= tree.predict(X_val_1)
print(classification_report(y_val, y_pred_1))


print("for columns with importance over 0.05")
tree = DecisionTreeClassifier()
tree.fit(X_train_2,y_train)
y_pred_2= tree.predict(X_val_2)
print(classification_report(y_val, y_pred_2))


print("for RAM only")
tree = DecisionTreeClassifier()
tree.fit(X_train_3.values.reshape(-1, 1), y_train)
y_pred_3= tree.predict(X_val_3.values.reshape(-1, 1))
print(classification_report(y_val, y_pred_3))

for all the columns
              precision    recall  f1-score   support

           0       0.95      0.85      0.90        73
           1       0.69      0.80      0.74        74
           2       0.73      0.71      0.72        73
           3       0.89      0.88      0.88        74

    accuracy                           0.81       294
   macro avg       0.82      0.81      0.81       294
weighted avg       0.82      0.81      0.81       294

for columns with importance over 0.02
              precision    recall  f1-score   support

           0       0.86      0.93      0.89        73
           1       0.75      0.69      0.72        74
           2       0.74      0.75      0.75        73
           3       0.90      0.89      0.90        74

    accuracy                           0.82       294
   macro avg       0.81      0.82      0.81       294
weighted avg       0.81      0.82      0.81       294

for columns with importance over 0.05
              precision    recall 